In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# to makes 'Dates' detected in dattime format, use 'parse_dates' option
df_train = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-1/train.csv', parse_dates=['Date'])
df_train.head()

In [ ]:
df_train.Date.max() - df_train.Date.min()

In [ ]:
df_train.Date.max(), df_train.Date.min()
date_diff = pd.to_datetime(df_train.Date.max()) - pd.to_datetime(df_train.Date.min())
date_diff

In [ ]:
# number of countries
len(df_train['Country/Region'].unique())

In [ ]:
# Cases > 0 first date
df_train['firstdate0'] = df_train.query('ConfirmedCases > 0').groupby('Country/Region')['Date'].transform('min')
# Cases > 20 first date
df_train['firstdate20'] = df_train.query('ConfirmedCases > 20').groupby('Country/Region')['Date'].transform('min')
# Cases > 50 first date
df_train['firstdate50'] = df_train.query('ConfirmedCases > 50').groupby('Country/Region')['Date'].transform('min')

In [ ]:
# day count since first date of confirmed case
df_train['daysSincefirstCase']  = (df_train['Date'] - df_train['firstdate0']).dt.days
df_train['daysSincefirstCase20']  = (df_train['Date'] - df_train['firstdate20']).dt.days
df_train['daysSincefirstCase50']  = (df_train['Date'] - df_train['firstdate50']).dt.days

In [ ]:
# plots
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(figsize= (20, 10))
# set title
ax.set_title('Number of Confirmed Cases since first reported count > 0')

for country, df in df_train.query('daysSincefirstCase > 0').groupby('Country/Region'):
    df_plot = df.set_index('daysSincefirstCase').sort_index()['ConfirmedCases'].cummax()
    df_plot.plot(label=country, ax=ax)
    ax.annotate(country, (df_plot.index[-1], df_plot.iloc[-1]))
ax.set_yscale('log')